In [42]:
import torch
import sqlite3
import pandas as pd
from dataset_to_data import *
dataset = MahjongDataset(data_path='database.db')


from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from Data_Skimmer import *

import time 
from sklearn.model_selection import train_test_split
import csv 
import torch.optim as optim
start_time = time.time()
X, y = [], []
for i in range (1,2000):
    temp = waits_data_proccessing(dataset[i])
    if(temp == None):
        continue
    X.append(temp[0])
    y.append(temp[1])
X = np.array(X)
y = np.array(y)

# zipping data into a database
# Zip corresponding rows of X and y together
zipped_data = [(X[i].tolist(), y[i].tolist()) for i in range(X.shape[0])]

# Connect to SQLite database (or create it)
conn = sqlite3.connect('test_data.db')
cursor = conn.cursor()

# Create a table
cursor.execute('''
CREATE TABLE IF NOT EXISTS test_table (
    id INTEGER PRIMARY KEY,
    X_values BLOB,
    y_values BLOB
)
''')

# Insert the zipped data into the table
for i, (X_row, y_row) in enumerate(zipped_data):
    cursor.execute('INSERT INTO test_table (X_values, y_values) VALUES (?, ?)', (str(X_row), str(y_row)))

# Commit and close the connection
conn.commit()
conn.close()


X = torch.tensor(X, dtype=torch.float)  # Use long for input data
y = torch.tensor(y, dtype=torch.float)     # Use long for integer labels
# until here!!!

index1 = int(len(X)*0.8)
index2 = int(len(X)*0.9)
X_train = X[:index1]
X_val = X[index1:index2]
X_test = X[index2:]
y_train = y[:index1]
y_val = y[index1:index2]
y_test = y[index2:]



print(X.shape, y.shape)  # Check shapes of tensors
print(f"Data processing time: {time.time() - start_time:.2f}s")

torch.Size([485, 368]) torch.Size([485, 34])
Data processing time: 12.29s


In [44]:

# Block of extracting data from the database
import ast

# Connect to the SQLite database
conn = sqlite3.connect('test_data.db')
cursor = conn.cursor()

# Query the data
cursor.execute('SELECT X_values, y_values FROM test_table')
rows = cursor.fetchall()

# Process the data
X_extracted = []
y_extracted = []

for row in rows:
    X_row = ast.literal_eval(row[0])  # Convert string back to list
    y_row = ast.literal_eval(row[1])  # Convert string back to list
    X_extracted.append(X_row)
    y_extracted.append(y_row)

# Convert lists back to NumPy arrays or tensors if needed
import numpy as np

X_extracted = np.array(X_extracted)
y_extracted = np.array(y_extracted)

# Optionally, convert to tensors
X = torch.tensor(X_extracted, dtype=torch.float)
y = torch.tensor(y_extracted, dtype=torch.float)

# Close the connection
conn.close()
print(X)
print(y)

tensor([[2., 1., 0.,  ..., 0., 0., 0.],
        [2., 1., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [2., 0., 0.,  ..., 0., 0., 0.],
        [2., 0., 0.,  ..., 0., 0., 0.],
        [2., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [38]:
import tensorflow as tf
from transformers import  BertModel
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn
import torch
import random

class MahjongModel(nn.Module):
    def __init__(self):
        super(MahjongModel, self).__init__()
        # Ensure the embedding dimension is compatible
        #self.embedding = nn.Embedding(37, 128)
        self.projection = nn.Linear(368, 768)
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.fc2 = nn.Linear(768, 34)
        
    def forward(self, x_batch):
        # Convert categorical features to embeddings
        #x_emb = self.embedding(x_batch)  # Shape: (batch_size, seq_length, embedding_dim)
        
        # Project concatenated features to BERT's hidden size
        #x = self.fc1(torch.cat((x_batch, x_emb)))  # Shape: (batch_size, hidden_size)
        #x = self.fc1(x_batch)
        # Add sequence length dimension for BERT
        #x = x.unsqueeze(1)  # Shape: (batch_size, sequence_length=1, hidden_size)
        x_batch = self.projection(x_batch)
        
        x_batch = x_batch.unsqueeze(1)
        # Use BERT to process the combined features
        outputs = self.bert(inputs_embeds=x_batch)[0]
        
        # Pass through classification head
        outputs = self.fc2(outputs)
        
        y_hat = torch.sigmoid(outputs)
        return y_hat


from torch.utils.data import Dataset 
class MahjongDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {
            'input': torch.tensor(self.data[idx], dtype=torch.float),
            'label': torch.tensor(self.labels[idx], dtype=torch.float)
        }
    
    
def getindices(tensor):
    mylist = []
    for i in range(len(tensor)):
        if tensor[i]:
            mylist.append(i)
    return mylist
    
    

def eval(model, X_val, y_val):
    model.eval()
    with torch.no_grad():
        outputs = model(X_val)
        outputs = (outputs >= 0.5).float()
        ran = random.randint(0, 1070)
        print(getindices(outputs[ran][0]),getindices(y_val[ran]))
        count = 0
        for i in range(y_val.size(0)):
            count += getindices(outputs[i][0]) == getindices(y_val[i])
        total_samples = y_val.size(0)
        misclassification_rate = 1.0 - (count / total_samples)
        return misclassification_rate

# Example Usage

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#device = "cpu"
model = MahjongModel().to(device)

train_dataset = MahjongDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)


loss_function = nn.BCELoss().to(device)
optimizer = Adam(model.parameters() ,lr=1e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2)

  
for epoch in range(15):
    model.train()
    num_batches = 0
    for param in model.bert.parameters():
        param.requires_grad = False
    epoch_loss = 0
    for batch in train_dataloader:
        inputs = batch["input"].to(device)
        labels = batch["label"].to(device)
        outputs = model(inputs)
        y_pred_tf = torch.squeeze(outputs).to(device)

        
        loss = loss_function(y_pred_tf, labels)
        optimizer.zero_grad()
        loss.backward()
        num_batches += 1

        optimizer.step()
        
        epoch_loss += loss.item()  # Accumulate the loss
    print("Epoch ", epoch, " train loss is: ", epoch_loss/num_batches)
    miss = eval(model, X_val.to(device), y_val.to(device))
    print("miss is ", miss)
    


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Epoch  0  train loss is:  0.2907749398379521
[] [3, 6]
miss is  1.0
Epoch  1  train loss is:  0.19707432749324572
[] [22, 25]
miss is  1.0
Epoch  2  train loss is:  0.196144374883751
[] [13, 16]
miss is  1.0
Epoch  3  train loss is:  0.19609373453381337
[] [14, 17]
miss is  1.0
Epoch  4  train loss is:  0.1955728329801205
[] [11]
miss is  1.0
Epoch  5  train loss is:  0.1953856410479457
[] [10, 22]
miss is  1.0
Epoch  6  train loss is:  0.19523376575411475
[] [12]
miss is  1.0
Epoch  7  train loss is:  0.19517767296404642
[] [27, 30]
miss is  1.0
Epoch  8  train loss is:  0.19480729740126868
[] [19]
miss is  1.0
Epoch  9  train loss is:  0.19486845974363803
[] [29]
miss is  1.0
Epoch  10  train loss is:  0.19487275233056023
[] [11, 14]
miss is  1.0
Epoch  11  train loss is:  0.194345602924939
[] [11]
miss is  1.0
Epoch  12  train loss is:  0.1942223177523418
[] [20]
miss is  1.0
Epoch  13  train loss is:  0.1939599982646318
[] [15]
miss is  1.0
Epoch  14  train loss is:  0.193681750377

In [40]:
optimizer = Adam(model.parameters() ,lr=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2)
  
for epoch in range(100): # can train for as long as you want
    model.train()
    for param in model.bert.parameters():
        param.requires_grad = False
    for param in model.bert.encoder.layer[-5:].parameters():
        param.requires_grad = True
    num_batches = 0
    epoch_loss = 0
    for batch in train_dataloader:
        inputs = batch["input"].to(device)
        labels = batch["label"].to(device)
        outputs = model(inputs)
        y_pred_tf = torch.squeeze(outputs).to(device)
        loss = loss_function(y_pred_tf, labels)
        optimizer.zero_grad()
        loss.backward()
        num_batches += 1

        optimizer.step()
        
        epoch_loss += loss.item()  # Accumulate the loss
    print("Epoch ", epoch, " train loss is: ", epoch_loss/num_batches)
    miss = eval(model, X_val.to(device), y_val.to(device))
    print("miss is ", miss)  

/tmp/ipykernel_30508/2766732523.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input': torch.tensor(self.data[idx], dtype=torch.float),
/tmp/ipykernel_30508/2766732523.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'label': torch.tensor(self.labels[idx], dtype=torch.float)


Epoch  0  train loss is:  0.08780361892011529
[] [11]
miss is  0.980410447761194
Epoch  1  train loss is:  0.08638691429921243
[8, 32] [4, 7]
miss is  0.9794776119402985
Epoch  2  train loss is:  0.08452742246280816
[20, 23] [0, 3]
miss is  0.980410447761194
Epoch  3  train loss is:  0.08363919953043576
[21] [4, 25]
miss is  0.9794776119402985
Epoch  4  train loss is:  0.08259138923318413
[9] [9, 28]
miss is  0.9794776119402985
Epoch  5  train loss is:  0.08155723425757043
[2] [21, 24]
miss is  0.9841417910447762
Epoch  6  train loss is:  0.08078751575935729
[1, 4] [14, 17]
miss is  0.9757462686567164
Epoch  7  train loss is:  0.07975285753718539
[] [22]
miss is  0.9757462686567164
Epoch  8  train loss is:  0.07751960798987226
[22, 25] [18, 21]
miss is  0.9748134328358209
Epoch  9  train loss is:  0.07657660102146266
[] [22, 26]
miss is  0.9813432835820896
Epoch  10  train loss is:  0.0767073558719628
[] [14, 17]
miss is  0.9785447761194029
Epoch  11  train loss is:  0.0748224569325331

KeyboardInterrupt: 